In [1]:
!pip install ultralytics
!pip install -U ipywidgets
from ultralytics import YOLO

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

model.train(
    data='/kaggle/input/cv-project-files2/dataset.yaml',
    epochs=75,                            # Number of training epochs
    imgsz=640,                            # Image size
    batch=32,                             # batch size
    lr0=0.001,                             # initial learning rate
    augment=True,                         # no data augmentation
    dropout=0.2       ,                    # dropout to reduce overfitting
    device=[0,1]
)


In [ ]:
#from ultralytics import YOLO
# resume training
#model = YOLO('/kaggle/working/runs/detect/train/weights/last.pt')
#model.train(resume=True)

In [4]:
image_folder = '/kaggle/input/large-license-plate-dataset/images/test/3241f09a8964ddfb.jpg'
#model = YOLO('/kaggle/working/runs/detect/train/weights/best.pt')
model = YOLO('/kaggle/input/cv-project-files2/best75.pt')

results = model(image_folder,stream=False)
print("HELLO")
i = 0
for image in results:
    for i in range(len(image)):
        """print(image.boxes)
        if i < 5:
            i+=1
            if image.boxes.xywh[box][2] > 50:
                box.save_crop('/kaggle/working/crop')
                print("Size ok")
            else:
                print("Too small")"""
        x, y, w, h = image.boxes.xywh[i].tolist() 

        if w > 50 and h > 50:
            box.save_crop(output_folder)
            print("Size ok, cropped and saved")
        else:
            print("Too small, skipping")


image 1/1 /kaggle/input/large-license-plate-dataset/images/test/3241f09a8964ddfb.jpg: 480x640 2 license_plates, 6.6ms
Speed: 2.1ms preprocess, 6.6ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)
HELLO
Too small, skipping
Too small, skipping


In [ ]:
# to save the run : zip the folder from this command and then download the zip file
#import shutil
#shutil.make_archive("train_20", 'zip', "/kaggle/working/runs/detect/train")